In [11]:
#not execute this part (from chatgpt), the code from AJ' seems to work fine at this moment.
from transformers import AutoModel, AutoTokenizer
import torch

# Load the pre-trained model
model_name = "bert-base-uncased"  # Replace with your original pre-trained model
model = AutoModel.from_pretrained(model_name)

# Load the tokenizer (if you added custom tokens, make sure to use your saved tokenizer)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load your adapter and optimizer states from checkpoint
checkpoint_dir = "path_to_your_checkpoint_directory"

# Load adapter weights (Safetensors)
model.load_state_dict(torch.load(f"{checkpoint_dir}/adapter_model.safetensors"))

# If you want to resume training:
#optimizer = torch.load(f"{checkpoint_dir}/optimizer.pt")
#scheduler = torch.load(f"{checkpoint_dir}/scheduler.pt")

# Load added tokens
#tokenizer.add_special_tokens({"additional_special_tokens": ["<token_1>", "<token_2>"]})

# Model is ready for inference
model.eval()

# Example inference
input_text = "Sample input text for inference"
inputs = tokenizer(input_text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

print(outputs)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-11-34b65e876c8f>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unle

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_checkpoint_directory/adapter_model.safetensors'

In [1]:
%pip install wandb psutil GPUtil

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=d579a01be6b747785983f442770e813c405a2c068eebf38d5cfc358c21535ae0
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


In [2]:
import wandb

# Initialize wandb for inference monitoring
wandb.init(project="huggingface", name="kdai-ftthgpt-2409232340-2409240031")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import psutil
import GPUtil
import time

# Function to log system metrics
def log_system_metrics():
    # CPU and Memory usage
    cpu_usage = psutil.cpu_percent()
    mem_usage = psutil.virtual_memory().percent
    # GPU usage (if available)
    gpus = GPUtil.getGPUs()
    if gpus:
        gpu_usage = gpus[0].load * 100
        gpu_memory = gpus[0].memoryUtil * 100
    else:
        gpu_usage = None
        gpu_memory = None

    # Log metrics to wandb
    wandb.log({
        "cpu_usage": cpu_usage,
        "memory_usage": mem_usage,
        "gpu_usage": gpu_usage if gpu_usage is not None else 0,
        "gpu_memory_usage": gpu_memory if gpu_memory is not None else 0
    })

In [4]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "openthaigpt/openthaigpt-1.0.0-beta-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.27G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [6]:
from huggingface_hub import login

# Log in to Hugging Face Hub
login()  # This will prompt you to enter your Hugging Face token


In [7]:
#test save model
#model.save_pretrained("./my_fine_tuned_model")
#tokenizer.save_pretrained("./my_fine_tuned_model")

model.save_pretrained('kdai-llm-lab-q4-ft-openthaigpt100b-13b-2409232340')
#tokenizer.save_pretrained('kdai-llm-lab-q4-ft-openthaigpt100b-13b-2409232340')

model.push_to_hub('ekwongsaman/kdai-llm-lab-q4-ft-openthaigpt100b-13b-2409232340')
#tokenizer.push_to_hub('ekwongsaman/kdai-llm-lab-q4-ft-openthaigpt100b-13b-2409232340')

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ekwongsaman/kdai-llm-lab-q4-ft-openthaigpt100b-13b-2409232340/commit/24aaa06d58376ffcbca925519bb09d9b532dbbca', commit_message='Upload LlamaForCausalLM', commit_description='', oid='24aaa06d58376ffcbca925519bb09d9b532dbbca', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/718k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.47M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

In [9]:


custom_prompt = "โปรดอธิบายลักษณะช่องปากที่เป็นมะเร็งในระยะเริ่มต้น"


PROMPT =f'[INST] <You are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<>{custom_prompt}[/INST]'



In [10]:
# Log system metrics
log_system_metrics()

start_time = time.time()

In [11]:
%%time
from peft import PeftModel
# from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from transformers import GenerationConfig

model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/llm_ThaiMed_checkpoints/thai-med-pack/checkpoint-2000")

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")

# Perform inference using mixed precision
with torch.autocast("cuda"):

  generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configurati

Generating...
<s> [INST] <You are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<>โปรดอธิบายลักษณะช่องปากที่เป็นมะเร็งในระยะเริ่มต้น[/INST]สวัสดี ฉันหวังว่านี่จะเป็นคำถามของคุณ ฉันเป็นแพทย์โรคผิวหนัง ฉันจะพยายามตอบคำถามของคุณ อาการของโรคฟันเป็นปัญหาที่ทำให้เกิดอาการปวด ในกรณีที่รุนแรง อาจมีอาการปวดเล็กน้อยถึงมากและตุ่มขนาดเล็กหรือขนาดใหญ่ที่มีอาการปวด สามารถรักษาด้วยยาปฏิชีวนะได้หลายชนิด เช่น การฉีดยาแก้ปวดและการใช้ยาปฏิชีว การผ่าตัดอาจต้องทำเพื่อกำจัดเนื้อร้ายออกไป มีโอกาสหายได้หากตรวจพบและรักษาตั้งแต่แรก ปัจจุบันนี้เทคโนโลยีการถ่ายภาพรังสีขั้นสูงช่วยในการวินิจฉัยและรักษาให้ออกมาดีที่สุด หวังว่านี่จะตอบคำถามของคุณ หากคุณมีข้อสงสัยเพิ่มเติม โปรดอย่าลังเลที่จะถาม ความนับถือ.</s>
CPU times: user 27.6 s, sys: 846 ms, total: 28.4 s
Wall time: 38.1 s


In [12]:
# Log system metrics
#log_system_metrics()

# Optionally, log other custom metrics such as inference time
#start_time = time.time()
#model_output = model(input_data)
end_time = time.time()
inference_time = end_time - start_time

# Log custom metrics to wandb
wandb.log({"inference_time": inference_time})
wandb.finish()

cpu_usage,▁
gpu_memory_usage,▁
gpu_usage,▁
inference_time,▁
memory_usage,▁
cpu_usage,4.6
gpu_memory_usage,35.11765
gpu_usage,0
inference_time,117.2074
memory_usage,5.1
